In [104]:
import carla
import numpy as np
import cv2
import time
import sys
import queue
from ultralytics import YOLO
import math
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [105]:
client = carla.Client('localhost', 2000)

world = client.get_world()

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]

setting = world.get_settings()
setting.fixed_delta_seconds = 1
setting.synchronous_mode = True
world.apply_settings(setting)


3645

In [107]:
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*mini*')
bicycle_bp = world.get_blueprint_library().filter('*crossbike*')


vehicle_start_point = spawn_points[1]
bicycle_start_point = spawn_points[1]

bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=10, y=-6), carla.Rotation(yaw=90))
bicycle.set_transform(bicyclepos)


vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)



In [108]:
CAMERA_POS_Z = 1.5 
CAMERA_POS_X = 0
CAMERA1_POS_Y = 0.5
CAMERA2_POS_Y = -0.5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')


camera1_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X, y = CAMERA2_POS_Y))
camera1 = world.spawn_actor(camera_bp,camera1_init_trans,attach_to=vehicle)


image_queue = queue.Queue()
camera1.listen(image_queue.put)

model = YOLO("best.pt")

In [109]:
def process_image(image):
    # Convert to numpy array and change from BGRA to BGR
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # CARLA RGB image is in BGRA format
    array = array[:, :, :3]  # Remove alpha channel
    return array

In [110]:
def process_collision(event):
    # Extract collision data
    global collision_happened
    other_actor = event.other_actor
    impulse = event.normal_impulse
    collision_location = event.transform.location
    print(f"Collision with {other_actor.type_id}")
    print(f"Impulse: {impulse}")
    print(f"Location: {collision_location}")
    collision_happened = True

In [111]:
collision_detector_bp = world.get_blueprint_library().find('sensor.other.collision')
collision_sensor = world.spawn_actor(
        collision_detector_bp,
        carla.Transform(),
        attach_to=vehicle
    )
collision_sensor.listen(lambda event: process_collision(event))

In [112]:
while True:
    vehicle.apply_control(carla.VehicleControl(throttle=0.5))
    bicycle.apply_control(carla.VehicleControl(throttle=1))
    world.tick()
    
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2+v.z**2), 0)
    print(f"Speed: {speed}")
    frame1 = image_queue.get()
    frame1 = process_image(frame1)
    #frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGRA2BGR)

    results = model(frame1, verbose=False)
    frame_copy = frame1.copy()

    for result in results:
        for box in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box.xyxy[0]
            center_x = int((x1 + x2) / 2)  # x-center of the bicycle
            center_y = int((y1 + y2) / 2)  # y-center of the bicycle
            #bicycles_left.append((center_x, center_y))
            conf = box.conf[0]            # Confidence score
            cls = box.cls[0]
            cv2.rectangle(frame_copy, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frame_copy, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)



    cv2.imshow('Camera1',frame_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


camera1.stop()
camera1.destroy()
vehicle.destroy()
cv2.destroyAllWindows()

Speed: 4.0
Speed: 7.0
Speed: 11.0
Speed: 3.0
Speed: 1.0
Speed: 1.0
Speed: 1.0
Speed: 0.0
Speed: 1.0
Speed: 4.0
Speed: 4.0
Speed: 5.0
Speed: 5.0
Speed: 6.0
Speed: 6.0
Speed: 6.0
Speed: 7.0
Speed: 7.0
Speed: 8.0
Speed: 8.0
Speed: 8.0
Speed: 9.0
Speed: 9.0
Speed: 9.0
Speed: 9.0
Speed: 10.0
Speed: 10.0
Speed: 10.0
Speed: 11.0
Speed: 11.0
Speed: 11.0
Speed: 11.0
Speed: 11.0
Speed: 12.0
Speed: 12.0
Speed: 12.0
Speed: 13.0
Speed: 13.0
Speed: 13.0
Speed: 13.0
Collision with vehicle.bh.crossbike
Impulse: Vector3D(x=76.126701, y=-28.444582, z=55.430759)
Location: Location(x=-59.593319, y=27.984507, z=0.042598)
Speed: 13.0
Collision with vehicle.bh.crossbike
Impulse: Vector3D(x=223.202469, y=-147.704620, z=6.930445)
Location: Location(x=-59.219509, y=27.985929, z=0.042538)
Speed: 13.0
Collision with vehicle.bh.crossbike
Impulse: Vector3D(x=23.588861, y=-15.359425, z=0.853276)
Location: Location(x=-58.845932, y=27.991945, z=0.043350)
Speed: 14.0
Collision with vehicle.bh.crossbike
Impulse: Vector3

In [113]:
#Destroying the existing things
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
world.tick()

3757